<a href="https://colab.research.google.com/github/gowriks12/student-attentiveness-classification/blob/main/student_attention_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installs
!pip install imageai
!pip install tensorflow
!pip install pickle-mixin
!pip install mediapipe
!pip install scikit-learn==1.0
# !python -m pip uninstall matplotlib
!pip install matplotlib==3.1.3
!pip install tensorflow-object-detection-api
!pip install dataframe-image
!pip install python-docx
!pip install pydub flask_ngrok flask_cors
!pip install shutil
!apt-get install xattr > /dev/null

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Importing Module
!pip install pydrive                             # Package to use Google Drive API - not installed in Colab VM by default
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

from google.colab import auth                    # Other necessary packages
from oauth2client.client import GoogleCredentials

auth.authenticate_user()                         # Follow prompt in the authorization process
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()

drive = GoogleDrive(gauth)
# https://drive.google.com/file/d/1UXpKfXMFVgNYDH5j3K-RZkWLTbGLKblv/view?usp=sharing
your_module = drive.CreateFile({"id": "1UXpKfXMFVgNYDH5j3K-RZkWLTbGLKblv"})   # "your_module_file_id" is the part after "id=" in the shareable link
your_module.GetContentFile("PoseModule.py")          # Save the .py module file to Colab VM

In [ ]:
# Imports
from imageai.Detection import ObjectDetection
import os
import pickle
from sklearn import preprocessing
import numpy as np
from google.colab.patches import cv2_imshow
import cv2
from skimage.color import rgb2gray
from skimage.transform import resize
import matplotlib.pyplot as plt
import math
import PoseModule as pm
import pandas as pd
import csv
from object_detection.utils import visualization_utils as vis_utils
# from PyPDF2 import PdfFileWriter
from PIL import Image
import matplotlib.pyplot as plt
# import pandas as pd
# from pandas.table.plotting import table
from pandas.plotting import table 
execution_path = "/content/drive/MyDrive/Colab Notebooks"
download_path = "/content/drive/MyDrive/Colab Notebooks/downloads"
upload_path = "/content/drive/MyDrive/Colab Notebooks/uploads"
%matplotlib inline
import dataframe_image as dfi
import docx
from docx.shared import Inches
from subprocess import getoutput
from IPython.display import HTML

In [ ]:
# Pose Classification Module
def multiPoseClassifier():
  detector = ObjectDetection()
  detector.setModelTypeAsRetinaNet()
  
  detector.setModelPath( os.path.join(execution_path , "resnet50_coco_best_v2.1.0.h5"))
  detector.loadModel()
  poseDetector = pm.PoseDetector()
  # poseDetector.poseClassifier()
  # loaded_model = pickle.load(open(os.path.join(execution_path , 'knn_r_pca_pipe.sav'), 'rb'))
  loaded_model = pickle.load(open(os.path.join(execution_path , 'rf_pipe_pickle.sav'), 'rb'))
  custom_objects = detector.CustomObjects(person=True)
  # folder_path = '/content/drive/MyDrive/Colab Notebooks/Thesis-object-detection/frames'

  output = []
  count = 0
  one_frame_each = 100
  folder_name = "test2"
  videofile = os.path.join(upload_path,"test.mov")
  vidcap = cv2.VideoCapture(videofile)

  success = True
  while success:
      if (count % one_frame_each == 0):                                   # checks frame number and keeps one_frame_each          
          success,image = vidcap.read() 
          print(count)
          # image = cv2.imread(os.path.join(folder_path , filename))
          file_name = 'frame' + str(count)+ '.png'
          cv2.imwrite(os.path.join(execution_path, folder_name, file_name), image)
          op_img = "temp.jpg"                                # reads next frame           
          detections = detector.detectCustomObjectsFromImage(custom_objects=custom_objects, input_image= os.path.join(execution_path, folder_name, file_name), output_image_path=os.path.join(execution_path , op_img), minimum_percentage_probability=30)
          person_count = 0
          img = Image.open(os.path.join(execution_path , folder_name, file_name))
          for eachObject in detections:
            person_count += 1
            if eachObject["percentage_probability"] > 65:
              boxPoints = eachObject["box_points"]
              x1, y1, x2, y2 = boxPoints
              top_left_x = min([x1,x2])
              top_left_y = min([y1,y2])
              bot_right_x = max([x1,x2])
              bot_right_y = max([y1,y2])
              cropped = image[top_left_y:bot_right_y+1, top_left_x:bot_right_x+1]
              # crop,className = poseDetector.poseClassifier(cropped, loaded_model)
              crop,className,confidence = poseDetector.poseClassifier(cropped,loaded_model)
              if confidence is not None:                
                confidence = max(confidence[0]) * 100
                confidence = round(confidence,2)               
              row = [count, person_count, eachObject["percentage_probability"], className,confidence]
              output.append(row)                
              op_str = ""
              op_str += str(className) + ":" + str(confidence) + "%"
          
              # print(className, confidence)
              vis_utils.draw_bounding_box_on_image(
                          img, top_left_y, top_left_x, bot_right_y, bot_right_x,
                          color='cyan',thickness=5,
                          # display_str_list=[str(className),":",str(confidence),"%"],
                          display_str_list=[op_str],
                          use_normalized_coordinates=False
                  )    
              # op = filename
              rgb_im = img.convert('RGB')
              rgb_im.save(os.path.join(execution_path, folder_name, file_name))   
      else:
        success,image = vidcap.read()
        # print("err")
          # success,image = vidcap.read()                                 # reads next frame    
      count += 1

  print(count)
  return output

In [ ]:
# Data Aggregation
def dataAnalyse(df):  
  df = df.dropna()  
  mapping = {
    "Hand_on_chin" : "Intermediate1",
    "Hand_on_head" : "Intermediate1",
    "Hand_raised" : "Attentive",
    "Hands_Crossed" : "Attentive",
    "Leaning_Forward" : "Intermediate2",
    "Looking_down" : "Intermediate2",
    "Looking_to_the_side" : "Inattentive",
    "Sitting_Straight" : "Attentive",
    "Sleeping" : "Inattentive",
    "Taking_Notes" : "Attentive"
  }
  df['Classification'] = df['Pose'].map(mapping) 

  # Classification percentage dataframe
  class_data = pd.DataFrame(df['Classification'].value_counts(normalize=True) * 100)
  dfi.export(class_data, os.path.join(download_path,'ClassTable.png'),table_conversion = 'matplotlib')
  # Bar plot
  ax = class_data.plot.barh(rot=0, figsize=(9, 7)).get_figure()
  ax.savefig(os.path.join(download_path,'classBar.png'))  
  # Pie Chart
  plot_image = class_data.plot.pie(subplots=True, figsize=(7, 7), autopct='%1.0f%%',legend = False)[0].get_figure()   
  plot_image.savefig(os.path.join(download_path,'classPie.png'))
  
  # Pose percentage dataframe
  pose_data = pd.DataFrame(df['Pose'].value_counts(normalize=True) * 100)
  dfi.export(pose_data, os.path.join(download_path,'PoseTable.png'),table_conversion = 'matplotlib')
  # Bar plot
  ax_p = pose_data.plot.barh(rot=0, figsize=(12, 9)).get_figure() 
  ax_p.savefig(os.path.join(download_path,'poseBar.png'))  
  # Pie Chart
  plot_image_p = pose_data.plot.pie(subplots=True, figsize=(9, 9), autopct='%1.0f%%',legend = False)[0].get_figure()
  plot_image_p.savefig(os.path.join(download_path,'posePie.png')) 
  

In [ ]:
def createDoc():
  
  # Create an instance of a word document
  doc = docx.Document()
    
  # Add a Title to the document
  doc.add_heading('Attentiveness Feedback Report', 0)
    
  # Image in its native size
  doc.add_heading('Classification Pie Chart', 3)
  doc.add_picture(os.path.join(download_path,'classPie.png'), width=Inches(4), height=Inches(4))

  doc.add_heading('Classification Bar Chart', 3)
  doc.add_picture(os.path.join(download_path,'classBar.png'), width=Inches(4), height=Inches(4))

  doc.add_heading('Classification table', 3)
  doc.add_picture(os.path.join(download_path,'ClassTable.png'), width=Inches(2), height=Inches(2))

  doc.add_heading('Posture Pie Chart', 3)
  doc.add_picture(os.path.join(download_path,'posePie.png'), width=Inches(4), height=Inches(4))

  doc.add_heading('Posture Bar Chart', 3)
  doc.add_picture(os.path.join(download_path,'poseBar.png'), width=Inches(4), height=Inches(4))

  doc.add_heading('Posture table', 3)
  doc.add_picture(os.path.join(download_path,'PoseTable.png'), width=Inches(4), height=Inches(4))
    
  # Now save the document to a location
  doc.save(os.path.join(download_path,'report.docx'))


In [ ]:
def createCSV(output):
  with open(os.path.join(download_path , 'op_CSV.csv'), mode='w') as classification_file:
      classification_writer = csv.writer(classification_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)
      classification_writer.writerow(["Frame No.", "Person No.", "Person Confidence", "Pose","Pose Confidence"])
      for row in output:
        classification_writer.writerow(row)

In [ ]:
# Main
# inp_video = "test2.mov"
# output = multiPoseClassifier(inp_video)
# createCSV(output)
# df = pd.DataFrame.from_records(output, columns = ["Frame No.", "Person No.", "Person Confidence", "Pose","Pose Confidence"])
# dataAnalyse(df)
# createDoc()



In [ ]:
from flask import Flask, jsonify, request
from flask_ngrok import run_with_ngrok
from flask import Flask, make_response, send_file
from flask_cors import CORS, cross_origin
from flask import jsonify
import shutil
import PIL
import os
from PIL import Image

app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'
app.config['UPLOAD_FOLDER'] = '/content/drive/MyDrive/Colab Notebooks/uploads'

run_with_ngrok(app)

 

@cross_origin()
@app.route('/', methods=['POST'])
def upload_file():
    uploaded_file = request.files['file']
    # print(uploaded_file.filename)
    # if os.path.exists("drive/MyDrive/Colab Notebooks/IncreaseAccuracy/Spectrograms/uploads") and os.path.isdir("drive/MyDrive/Colab Notebooks/IncreaseAccuracy/Spectrograms/uploads"):
    #   shutil.rmtree("drive/MyDrive/Colab Notebooks/IncreaseAccuracy/Spectrograms/uploads")
    if uploaded_file.filename != '':
        uploaded_file.save(os.path.join(app.config['UPLOAD_FOLDER'], "test.mov"))
    
    output = multiPoseClassifier()
    createCSV(output)
    df = pd.DataFrame.from_records(output, columns = ["Frame No.", "Person No.", "Person Confidence", "Pose","Pose Confidence"])
    dataAnalyse(df)
    createDoc()        

    fid1 = getoutput("xattr -p 'user.drive.id' '/content/drive/My Drive/Colab Notebooks/downloads/classPie.png' ")
    fid2 = getoutput("xattr -p 'user.drive.id' '/content/drive/My Drive/Colab Notebooks/downloads/posePie.png' ")
    fid3 = getoutput("xattr -p 'user.drive.id' '/content/drive/My Drive/Colab Notebooks/downloads/classBar.png' ")
    fid4 = getoutput("xattr -p 'user.drive.id' '/content/drive/My Drive/Colab Notebooks/downloads/poseBar.png' ")
    fid5 = getoutput("xattr -p 'user.drive.id' '/content/drive/My Drive/Colab Notebooks/downloads/report.docx' ")
    response  = {"files":[fid1,fid2,fid3,fid4,fid5]}
    return response



if __name__ == "__main__":
  app.run()

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')  # access drive
# # need to install xattr

# # get the id
# fid = getoutput("xattr -p 'user.drive.id' '/content/drive/My Drive/Colab Notebooks/downloads/classBar.png' ")
# print(fid)
# print("1-_6lTGybYp3EToNPa3Kt2rmH1GL6evKy")
# # make a link and display it
# HTML(f"<a href=https://colab.research.google.com/drive/{fid} target=_blank>notebook</a>")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
1-_6lTGybYp3EToNPa3Kt2rmH1GL6evKy
1-_6lTGybYp3EToNPa3Kt2rmH1GL6evKy


In [ ]:
# import pandas as pd
# import os
# execution_path = "/content/drive/MyDrive/Colab Notebooks"
# op = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/res.csv')
# # ct, pt = dataAnalyse(op)
# pie_class , bar_class, pie_pose, bar_pose, ct, pt = dataAnalyse(op)
# # type(pie_class)

# f_img = "drive/MyDrive/Colab Notebooks/IncreaseAccuracy/Spectrograms/uploads/upload.png"
    # img1 = Image.open(os.path.join(download_path,'classPie.png'))
    # img2 = Image.open(os.path.join(download_path,'classBar.png'))
    # img3 = Image.open(os.path.join(download_path,'posePie.png'))
    # img4 = Image.open(os.path.join(download_path,'poseBar.png'))

    # img_list = [img1, img2, img3, img4]
    # doc = os.path.join(download_path,'report.docx')

    # # return send_file(os.path.join(download_path,'classPie.png'), mimetype='image/png')

    # ##reuslt  contains list of path images
    # result = [os.path.join(download_path,'classPie.png'),os.path.join(download_path,'classBar.png'),os.path.join(download_path,'posePie.png'),os.path.join(download_path,'poseBar.png')]
    # encoded_imges = []
    # for image_path in result:
    #     encoded_imges.append(get_response_image(image_path))
    # return jsonify({'result': encoded_imges})